In [1]:
# =====================================================================
# Approach 1 (bubble → non-bubble): TRAINING on 4 bubble CSVs
# - Keeps your pipeline identical; swaps encoder via ENCODER_KIND flag.
# - Default: Transformer (ablation). Set "bilstm" to reproduce baseline.
# =====================================================================

# 0) CONFIG
ENCODER_KIND = "transformer"    # {"transformer","bilstm"}
EMB          = 128              # capacity ~ parity with baseline
POOL         = "last"           # {"last","mean","cls"}; mirrors BiLSTM last hidden (baseline pooling uncertain)
LR           = 3e-4
EPOCHS       = 300
TEMPERATURE  = 0.05
BATCH_CAP    = 64
SEED         = 42

# 1) Upload & Save (select EXACTLY 4 bubble merged_*.csv files)
from google.colab import files
import pathlib, glob, os, warnings, re, collections, random, math
warnings.filterwarnings("ignore")
import numpy as np
import torch

def set_seed(s=SEED):
    import random
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(SEED)

DATA_DIR = "/content/merged_csvs"
pathlib.Path(DATA_DIR).mkdir(exist_ok=True)

uploaded = files.upload()  # ★ Upload exactly 4 bubble training CSVs
for n, d in uploaded.items():
    with open(f"{DATA_DIR}/{n}", "wb") as f:
        f.write(d)
print("📝 Saved:", glob.glob(f"{DATA_DIR}/*.csv"))

# 2) Dedup names + PPIACO→PPI + required columns validation
import pandas as pd

raw_paths = glob.glob(os.path.join(DATA_DIR, "*.csv"))
dedup = collections.OrderedDict()
for p in sorted(raw_paths):  # alphabetical; push "(1)" copies behind
    key = re.sub(r"\s*\(\d+\)", "", os.path.basename(p))
    if key not in dedup:
        dedup[key] = p
paths = list(dedup.values())

need_cols  = ["Date","CPI","PPI","FEDFUNDS","DGS10","DJIA","SP500_PE"]
macro_cols = ["CPI","PPI","FEDFUNDS","DGS10"]
dow_cols   = ["DJIA","SP500_PE"]

assert len(paths) == 4, f"❗ Found {len(paths)} CSVs — need exactly 4 bubble files!"

# Ensure PPI column name
for p in paths:
    df_tmp = pd.read_csv(p)
    if "PPIACO" in df_tmp.columns and "PPI" not in df_tmp.columns:
        df_tmp.rename(columns={"PPIACO":"PPI"}, inplace=True)
        df_tmp.to_csv(p, index=False)

# Validate columns
for p in paths:
    miss = set(need_cols) - set(pd.read_csv(p, nrows=1).columns)
    assert not miss, f"{os.path.basename(p)} missing columns: {miss}"
print("✅ 4 bubble files OK:", [os.path.basename(p) for p in paths])

# 3) Libraries & Preprocessing
!pip -q install tsaug
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tsaug import TimeWarp, Drift, AddNoise

DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
WINDOW  = 24  # 24-month windows

# Load & concat (tag each as Prototype)
df_list = []
for p in paths:
    tmp = pd.read_csv(p, parse_dates=["Date"])
    if "PPIACO" in tmp.columns and "PPI" not in tmp.columns:
        tmp.rename(columns={"PPIACO":"PPI"}, inplace=True)
    proto = os.path.basename(p).replace(".csv","")
    tmp["Prototype"] = proto
    df_list.append(tmp)

df = (pd.concat(df_list, ignore_index=True)
        .sort_values("Date")
        .dropna(subset=need_cols)
        .reset_index(drop=True))

# Scale macro & market columns (fit on training only)
sc_macro = StandardScaler().fit(df[macro_cols])
sc_dow   = StandardScaler().fit(df[dow_cols])
Xm_all   = sc_macro.transform(df[macro_cols]).astype("float32")
Xd_all   = sc_dow.transform(df[dow_cols]).astype("float32")
prot_codes = pd.Categorical(df["Prototype"]).codes

print(f"📊 Training prototypes (4): {df['Prototype'].unique()}")
print(f"📊 Total rows after clean: {len(df)}")

# 4) Contrastive Dataset & DataLoader (unchanged)
aug = TimeWarp(n_speed_change=3, max_speed_ratio=2.0) + \
      Drift(max_drift=(0,0.1)) + \
      AddNoise(scale=0.01)

class ContrastiveDataset(Dataset):
    def __init__(self, Xm, Xd, codes, win=24):
        self.Xm, self.Xd, self.codes, self.win = Xm, Xd, codes, win
        # window starts that do not cross prototype boundaries
        self.starts = [i for b in np.unique(codes)
                       for i in np.where(codes==b)[0][:-win]
                       if i+win in np.where(codes==b)[0]]

    def __len__(self): return len(self.starts)

    def __getitem__(self, idx):
        s = self.starts[idx]
        anc = np.hstack([self.Xm[s:s+self.win], self.Xd[s:s+self.win]])
        alt = np.hstack([aug.augment(self.Xm[s:s+self.win]),
                         aug.augment(self.Xd[s:s+self.win])])
        return torch.tensor(anc), torch.tensor(alt)

train_ds   = ContrastiveDataset(Xm_all, Xd_all, prot_codes, WINDOW)
batch_size = min(BATCH_CAP, len(train_ds))
loader     = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
print(f"train_ds={len(train_ds)}, batch_size={batch_size}, len(loader)={len(loader)}")

# 5) Encoders + Head (BiLSTM baseline & Transformer ablation)
class EncoderBiLSTM(nn.Module):
    def __init__(self, in_dim, emb=128):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, emb, num_layers=2, bidirectional=True, batch_first=True)
        self.fc   = nn.Linear(emb*2, emb)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = torch.cat([h[-2], h[-1]], dim=1)
        z = self.fc(h)
        return nn.functional.normalize(z, dim=1)

class SinusoidalPE(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderTransformer(nn.Module):
    def __init__(self, in_dim, emb=128, nhead=4, num_layers=2, dropout=0.1, pool="last"):
        super().__init__()
        self.input = nn.Linear(in_dim, emb)
        layer = nn.TransformerEncoderLayer(d_model=emb, nhead=nhead, batch_first=True,
                                           dropout=dropout, norm_first=True)
        self.enc = nn.TransformerEncoder(layer, num_layers=num_layers)
        self.pos = SinusoidalPE(emb)
        self.pool = pool
        self.cls = nn.Parameter(torch.zeros(1,1,emb)) if pool == "cls" else None
    def forward(self, x):
        h = self.input(x)
        if self.cls is not None:
            cls = self.cls.expand(x.size(0), -1, -1)
            h = torch.cat([cls, h], dim=1)
        h = self.enc(self.pos(h))
        if self.pool == "cls":   z = h[:, 0, :]
        elif self.pool == "mean": z = h.mean(dim=1)
        else:                    z = h[:, -1, :]
        return nn.functional.normalize(z, dim=1)

class BubbleDetector(nn.Module):
    def __init__(self, in_dim, emb=128, kind="bilstm", pool="last",
                 nhead=4, num_layers=2, dropout=0.1):
        super().__init__()
        if kind == "bilstm":
            self.encoder = EncoderBiLSTM(in_dim, emb)
        elif kind == "transformer":
            self.encoder = EncoderTransformer(in_dim, emb, nhead, num_layers, dropout, pool)
        else:
            raise ValueError(f"Unknown encoder kind: {kind}")
        self.classifier = nn.Sequential(
            nn.Linear(emb, 64), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(32, 1), nn.Sigmoid()
        )
    def forward(self, x):
        z = self.encoder(x)
        prob = self.classifier(z)
        return z, prob.squeeze()
    @torch.no_grad()
    def get_probability(self, x):
        _, prob = self.forward(x); return prob

in_dim = len(macro_cols) + len(dow_cols)   # 6 features
model = BubbleDetector(in_dim=in_dim, emb=EMB, kind=ENCODER_KIND, pool=POOL,
                       nhead=4, num_layers=2, dropout=0.1).to(DEVICE)

def count_params(m): return sum(p.numel() for p in m.parameters() if p.requires_grad)
print(f"🧠 Encoder: {ENCODER_KIND} | POOL={POOL} | EMB={EMB}")
print(f"🧮 Trainable params: {count_params(model):,}")

# 6) Training (NT-Xent + BCE), identical logic to your baseline
temperature = TEMPERATURE
def ntxent(z1, z2, tau=temperature):
    z1 = nn.functional.normalize(z1, dim=1)
    z2 = nn.functional.normalize(z2, dim=1)
    sim = torch.mm(z1, z2.t()) / tau
    labels = torch.arange(z1.size(0), device=z1.device)
    return nn.CrossEntropyLoss()(sim, labels)

bce_loss = nn.BCELoss()
opt = optim.Adam(model.parameters(), lr=LR)

print(f"\n🚀 Training start — all training windows labeled bubble=1")
for ep in range(1, EPOCHS+1):
    model.train()
    tot_loss = 0.0
    for anc, alt in loader:
        anc, alt = anc.to(DEVICE), alt.to(DEVICE)
        z1, p1 = model(anc)
        z2, p2 = model(alt)
        cont = ntxent(z1, z2)
        labels = torch.ones(anc.size(0), device=DEVICE)
        clas = bce_loss(p1, labels) + bce_loss(p2, labels)
        loss = cont + 0.5*clas
        opt.zero_grad(); loss.backward(); opt.step()
        tot_loss += loss.item()
    if ep % 10 == 0 or ep == 1:
        print(f"Epoch {ep:03d} | loss {tot_loss/max(1,len(loader)):.4f}")

# 7) Save package (model + scalers + config)
package = {
    "model_state_dict": model.state_dict(),
    "model_config": {
        "encoder_kind": ENCODER_KIND,
        "emb": EMB,
        "pool": POOL,
        "window": WINDOW,
        "in_dim": in_dim,
        "transformer": {"nhead": 4, "num_layers": 2, "dropout": 0.1},
        "train_prototypes": list(pd.unique(df["Prototype"]))
    },
    "scalers": {
        "sc_macro": sc_macro,
        "sc_dow": sc_dow,
        "need_cols": need_cols,
        "macro_cols": macro_cols,
        "dow_cols": dow_cols
    },
    "param_count": int(count_params(model))
}

save_path = "bubble_model_package.pth"
torch.save(package, save_path)
print(f"✅ Saved: {save_path}")

# Optional auto-download (Colab)
try:
    from google.colab import files as colab_files
    colab_files.download(save_path)
except Exception:
    pass

print("🎯 Training done.")


Saving Merged_Black_Monday.csv to Merged_Black_Monday.csv
Saving Merged_Dot_Com.csv to Merged_Dot_Com.csv
Saving Merged_Nifty_Fifty.csv to Merged_Nifty_Fifty.csv
Saving Merged_Subprime_Bubble.csv to Merged_Subprime_Bubble.csv
📝 Saved: ['/content/merged_csvs/Merged_Subprime_Bubble.csv', '/content/merged_csvs/Merged_Black_Monday.csv', '/content/merged_csvs/Merged_Dot_Com.csv', '/content/merged_csvs/Merged_Nifty_Fifty.csv']
✅ 4 bubble files OK: ['Merged_Black_Monday.csv', 'Merged_Dot_Com.csv', 'Merged_Nifty_Fifty.csv', 'Merged_Subprime_Bubble.csv']
📊 Training prototypes (4): ['Merged_Nifty_Fifty' 'Merged_Black_Monday' 'Merged_Dot_Com'
 'Merged_Subprime_Bubble']
📊 Total rows after clean: 148
train_ds=52, batch_size=52, len(loader)=1
🧠 Encoder: transformer | POOL=last | EMB=128
🧮 Trainable params: 1,197,313

🚀 Training start — all training windows labeled bubble=1
Epoch 001 | loss 3.2306
Epoch 010 | loss 2.4212
Epoch 020 | loss 2.3091
Epoch 030 | loss 1.9929
Epoch 040 | loss 1.8937
Epoch 05

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎯 Training done.
